In [15]:
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


load_dotenv()

True

---
### Шаг 1. Установка LangChain и базовые сущности

In [9]:
llm  = ChatOllama(model="gpt-oss:120b-cloud", temperature=0.0)
response = llm .invoke("Какой сегодня день недели?")
print(response.model_dump_json())

{"content":"Сегодня — понедельник (3 ноября 2025 года).","additional_kwargs":{},"response_metadata":{"model":"gpt-oss:120b-cloud","created_at":"2025-11-03T15:15:29.486657957Z","done":true,"done_reason":"stop","total_duration":9677416390,"load_duration":null,"prompt_eval_count":73,"prompt_eval_duration":null,"eval_count":558,"eval_duration":null,"model_name":"gpt-oss:120b-cloud","model_provider":"ollama"},"type":"ai","name":null,"id":"lc_run--943f07b7-4750-4898-ad83-d044baf56afb-0","tool_calls":[],"invalid_tool_calls":[],"usage_metadata":{"input_tokens":73,"output_tokens":558,"total_tokens":631}}


---
### Шаг 2. Простейшая цепочка Prompt → LLM → Output

In [14]:
llm  = ChatOllama(model="gpt-oss:120b-cloud", temperature=1.0)

template = "Объясни простыми словами, что такое {topic}."
prompt = PromptTemplate(input_variables=["topic"], template=template)

chain = prompt | llm

result = chain.invoke(input="машинное обучение")

result

AIMessage(content='**Машинное обучение\u202f—\u202fэто способ научить компьютер решать задачи, не объясняя ему каждое правило вручную.**  \n\nПредставьте, что вы хотите, чтобы программа распознавала фотографии котов. Вместо того, чтобы прописывать всё, что делает кота «мягким», «имеет уши», «мяукает», вы показываете ей **много** примеров фотографий: некоторые с котами, некоторые без. Программа (алгоритм) ищет в этих данных закономерности‑«признаки», которые отличают одни картинки от других, и «обучается» на основе этих примеров. После обучения она уже может смотреть на новую, ранее не виденную фотографию и сказать, есть на ней кот или нет.\n\n### Как это работает в простых шагах\n\n| Шаг | Что делаем | Почему это важно |\n|-----|------------|------------------|\n| **1. Сбор данных** | Сбираем примеры (фотографии, тексты, измерения и т.п.). | Без данных «учиться» нечего. |\n| **2. Разметка** | Помечаем каждый пример тем, что он из себя представляет (кот / не кот, спам / не спам и т.д.).

In [13]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Объясни простыми словами, что такое {topic}.')
| ChatOllama(model='gpt-oss:120b-cloud', temperature=0.0)

---
#### Шаг 3. Добавление Memory (история диалога)


In [16]:
chat  = ChatOllama(model="gpt-oss:120b-cloud", temperature=0.0)

# Создаем пустой словарь для хранения истории сообщений по session_id
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """
    Функция для получения или создания истории по session_id.
    Каждый пользователь/разговор имеет свой session_id.
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты асистент, который хорошо разбирается в {ability}"),
    MessagesPlaceholder(variable_name="history"),  # Сюда будет вставлена история
    ("human", "{question}"),  # Текущий вопрос пользователя
])

chain = prompt | chat

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # Функция для получения истории
    input_messages_key="question",  # Ключ для текущего вопроса
    history_messages_key="history",  # Ключ для истории в промпте
)

In [17]:
response1 = chain_with_history.invoke(
    {"ability": "математика", "question": "Что такое косинус?"},
    config={"configurable": {"session_id": "user_123"}}
)
response1

AIMessage(content='**Косинус**\u202f—\u202fодна из основных тригонометрических функций. Он связывает угол в прямоугольном треугольнике или на единичной окружности с отношением сторон (или координатами).\n\n---\n\n## 1. Геометрическое определение (прямоугольный треугольник)\n\nДля угла \\( \\alpha \\) в прямоугольном треугольнике  \n\n\\[\n\\cos \\alpha = \\frac{\\text{длина прилежащего катета}}{\\text{длина гипотенузы}}.\n\\]\n\nТо есть косинус\u202f—\u202fотношение стороны, прилежащей к углу, к самой длинной стороне треугольника.\n\n---\n\n## 2. Определение через единичную окружность\n\nРассмотрим единичную окружность (радиус\u202f=\u202f1) с центром в начале координат. Для любого угла \\( \\theta \\) (измеряемого от положительного направления оси \\(x\\) против часовой стрелки) точка пересечения луча, образующего угол \\( \\theta \\), с окружностью имеет координаты  \n\n\\[\n(\\cos \\theta,\\; \\sin \\theta).\n\\]\n\nТаким образом, косинус\u202f—\u202fэто **\\(x\\)-координата** этой 

In [18]:
response2 = chain_with_history.invoke(
    {"ability": "математика", "question": "А как он связан с синусом?"},
    config={"configurable": {"session_id": "user_123"}}
)
response2

AIMessage(content='## Связь косинуса и синуса  \n\nКосинус\u202f—\u202fэто не отдельная «загадочная» функция, а **синус того же угла, но сдвинутый по фазе**. Ниже перечислены основные отношения, которые связывают \\(\\cos\\) и \\(\\sin\\).\n\n---\n\n### 1. Ко‑функциональное (ко‑угловое) тождество  \n\nДля любого угла \\(\\theta\\)\n\n\\[\n\\boxed{\\sin\\theta = \\cos\\!\\left(\\frac{\\pi}{2}-\\theta\\right)},\\qquad\n\\boxed{\\cos\\theta = \\sin\\!\\left(\\frac{\\pi}{2}-\\theta\\right)}.\n\\]\n\nТо есть синус угла равен косинусу его **дополнения** до \\(\\displaystyle \\frac{\\pi}{2}\\) (90°).  \nГеометрически это отражает тот факт, что в прямоугольном треугольнике катет, прилежащий к углу \\(\\theta\\), является противолежащим к углу \\(\\frac{\\pi}{2}-\\theta\\).\n\n---\n\n### 2. Пифагорово тождество  \n\n\\[\n\\boxed{\\sin^{2}\\theta + \\cos^{2}\\theta = 1}\\qquad(\\forall\\theta)\n\\]\n\nЭто следует из того, что точка \\((\\cos\\theta,\\sin\\theta)\\) лежит на единичной окружности 

In [19]:
response3 = chain_with_history.invoke(
    {"ability": "история", "question": "Кто был Александр Великий?"},
    config={"configurable": {"session_id": "user_456"}}
)
response3

AIMessage(content='**Александр\u202fIII Македонский (Александр Великий)** — один из самых известных полководцев и правителей древности, живший в IV\u202fвеке\u202fдо\u202fн.\u202fэ.  \n\n| Параметр | Описание |\n|---|---|\n| **Родился** | 20\u202fиюля\u202f356\u202fг. до\u202fн.\u202fэ. в Пелле, столице Македонского царства. |\n| **Семья** | Сын царя Филиппа\u202fII и Олимпиады (из Эпира). Был воспитан Аристотелем, который дал ему фундаментальное образование в философии, науке и культуре. |\n| **Взошёл на престол** | После убийства отца в 336\u202fг. до\u202fн.\u202fэ. (в 20 лет). |\n| **Главные военные кампании** | <ul><li>**Иранская (персидская) империя** – 334‑331\u202fгг.: битвы при Гранике, Иссе, Гавгамелах, где Александр одержал решающие победы над Дарием\u202fIII.</li><li>**Экспедиция в Индию** – 327‑325\u202fгг.: битва при Гидаспе, поход вдоль реки Инд, достижение реки Ганг.</li></ul> |\n| **Территория империи** | На пике своего могущества охватывала почти весь известный тогда 

In [24]:
response1.usage_metadata, response2.usage_metadata, response3.usage_metadata

({'input_tokens': 95, 'output_tokens': 1037, 'total_tokens': 1132},
 {'input_tokens': 1095, 'output_tokens': 1248, 'total_tokens': 2343},
 {'input_tokens': 95, 'output_tokens': 1335, 'total_tokens': 1430})

In [25]:
store

{'user_123': InMemoryChatMessageHistory(messages=[HumanMessage(content='Что такое косинус?', additional_kwargs={}, response_metadata={}), AIMessage(content='**Косинус**\u202f—\u202fодна из основных тригонометрических функций. Он связывает угол в прямоугольном треугольнике или на единичной окружности с отношением сторон (или координатами).\n\n---\n\n## 1. Геометрическое определение (прямоугольный треугольник)\n\nДля угла \\( \\alpha \\) в прямоугольном треугольнике  \n\n\\[\n\\cos \\alpha = \\frac{\\text{длина прилежащего катета}}{\\text{длина гипотенузы}}.\n\\]\n\nТо есть косинус\u202f—\u202fотношение стороны, прилежащей к углу, к самой длинной стороне треугольника.\n\n---\n\n## 2. Определение через единичную окружность\n\nРассмотрим единичную окружность (радиус\u202f=\u202f1) с центром в начале координат. Для любого угла \\( \\theta \\) (измеряемого от положительного направления оси \\(x\\) против часовой стрелки) точка пересечения луча, образующего угол \\( \\theta \\), с окружностью

In [26]:
history = get_session_history("user_123")
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")

human: Что такое косинус?
ai: **Косинус** — одна из основных тригонометрических функций. Он связывает угол в прямоугольном треугольнике или на единичной окружности с отношением сторон (или координатами).

---

## 1. Геометрическое определение (прямоугольный треугольник)

Для угла \( \alpha \) в прямоугольном треугольнике  

\[
\cos \alpha = \frac{\text{длина прилежащего катета}}{\text{длина гипотенузы}}.
\]

То есть косинус — отношение стороны, прилежащей к углу, к самой длинной стороне треугольника.

---

## 2. Определение через единичную окружность

Рассмотрим единичную окружность (радиус = 1) с центром в начале координат. Для любого угла \( \theta \) (измеряемого от положительного направления оси \(x\) против часовой стрелки) точка пересечения луча, образующего угол \( \theta \), с окружностью имеет координаты  

\[
(\cos \theta,\; \sin \theta).
\]

Таким образом, косинус — это **\(x\)-координата** этой точки.

---

## 3. Свойства и основные факты

| Свойство | Формулировка |
|-----